In [1]:
%pip install textacy
%pip install transformers
%pip install datasets

     |████████████████████████████████| 200 kB 8.4 MB/s 
     |████████████████████████████████| 2.0 MB 47.5 MB/s 
     |████████████████████████████████| 481 kB 14.4 MB/s 
     |████████████████████████████████| 6.0 MB 46.1 MB/s 
     |████████████████████████████████| 132 kB 49.8 MB/s 
     |████████████████████████████████| 653 kB 58.2 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 181 kB 69.4 MB/s 
     |████████████████████████████████| 457 kB 55.7 MB/s 
     |████████████████████████████████| 10.1 MB 53.8 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1230863 sha256=8695d6eecab938b724e98f0d7dce4c3227b850887d6902907dabb8fd9672faf0
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73968 sha256=1a3c680a125832a2b09c4c0234eb2a230d9ec121845bca1

**data sanitization:**

remove everything after "footnote", replace /n with ""

In [2]:
from datasets import load_dataset

train_data, test_data = load_dataset("rzhang123/UScourt", split =['train', 'test'])

Using custom data configuration rzhang123--UScourt-3db0629de3e01742


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/rzhang123--UScourt-3db0629de3e01742/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import AlbertTokenizerFast, AlbertForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

import torch

tokenizer = AlbertTokenizerFast.from_pretrained("albert-base-v2")
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=13)

def preprocess_function(examples):
    return tokenizer(examples['text'], padding = 'max_length', truncation=True)
  
train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [4]:
len(train_data['input_ids'][0])

512

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
  output_dir = '/results',
  num_train_epochs = 15,
  per_device_train_batch_size = 16,
  per_device_eval_batch_size= 16,
  evaluation_strategy = "epoch",
  save_strategy='epoch',
  disable_tqdm = False, 
  load_best_model_at_end=True,
  warmup_steps=200,
  weight_decay=0.01,
  logging_steps = 4,
  fp16 = True,
  logging_dir='/logs',
  dataloader_num_workers = 0,
  run_name = 'ALbert-classification'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics 
)

#device = 'cuda' if torch.cuda.is_available() else 'cpu'


Using amp half precision backend


In [7]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5624
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5280


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.100100,1.195171,0.631902,0.337169,0.360049,0.357228
2,0.874100,1.002472,0.679538,0.510024,0.614135,0.497250
3,0.775700,0.903962,0.716348,0.572319,0.675647,0.540719
4,0.505200,0.968103,0.694695,0.561031,0.657944,0.537487
5,0.670300,0.900674,0.731866,0.617107,0.653374,0.605772
6,0.275100,0.906235,0.743414,0.641211,0.658478,0.641395
7,0.237000,0.969898,0.734031,0.633632,0.643088,0.640148
8,0.255200,1.030286,0.738001,0.620780,0.656214,0.607425
9,0.158800,1.259350,0.741610,0.632979,0.666291,0.610759
10,0.095300,1.486124,0.738722,0.641045,0.647204,0.649057


The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2771
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /results/checkpoint-352
Configuration saved in /results/checkpoint-352/config.json
Model weights saved in /results/checkpoint-352/pytorch_model.bin
tokenizer config file saved in /results/checkpoint-352/tokenizer_config.json
Special tokens file saved in /results/checkpoint-352/special_tokens_map.json
The following columns in t

TrainOutput(global_step=5280, training_loss=0.4074748425165925, metrics={'train_runtime': 4739.6505, 'train_samples_per_second': 17.799, 'train_steps_per_second': 1.114, 'total_flos': 2018236047482880.0, 'train_loss': 0.4074748425165925, 'epoch': 15.0})